# CNNs for Heart Rate Estimation and Human Activity Recognition in Wrist Worn Sensing Applications


This is code for reproducing the HAR results shown in the paper presented at the WristSense workshop as part of PerCom 2020.

## Data Collection

The data was collected by [D. Jarchi and A. Casson (2017)](https://www.mdpi.com/2306-5729/2/1/1) and downloaded from [PhysioNet](https://physionet.org/content/wrist/1.0.0/).

### Using Google Colaboratory - (Recommended Working Environment)

You can run this notebook on Colab using the following cell to mount your drive and install some dependencies

You may need to install some of these packages below

In [0]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Install wfdb - loading the data using Python
!pip install wfdb

In [0]:
import wfdb
import os
import matplotlib.pyplot as plt
import numpy as np

# Change cwd if necessary
path = '/content/drive/My Drive/WristSense/CNNs_HAR_and_HR/'
os.chdir(path)

Make the directories to store the images

In [0]:
ex = ['High', 'Low', 'Run', 'Walk']
fs=256.0
freqs = [fs//256.0, fs//30.0, fs//15.0, fs//12.0, fs//11.0, fs//10.0, fs//9.0, fs//8.0, fs//5.0, fs//1.0]

for f in freqs:
  os.mkdir('./Data/PPG_'+str(int(fs//f))+'Hz')
  for e in ex:
    os.mkdir('./Data/PPG_'+str(int(fs//f))+'Hz/'+str(e))

## Load Data

This step is done in by selecting each exercise at a time. We can begin with the 'walk' exercise.

By changing the word below betwwen 'high', 'low', 'run', 'walk' we can pre-process our data.

In [0]:
def load_data(fileDir, exercise):
    word = exercise.lower()
    file_path_list = []
    valid_file_extensions = [".dat"]
    valid_file_extensions = [item.lower() for item in valid_file_extensions]


    for file in os.listdir(fileDir):
        extension = os.path.splitext(file)[1]
        if extension.lower() not in valid_file_extensions:
            continue
        file_path_list.append(os.path.join(fileDir, file))

    PPG = []
    for path in file_path_list:
        base=os.path.basename(path)
        base = os.path.splitext(base)[0]
        if word in base:
            print(fileDir+'/%s'%(base))
            sample = wfdb.rdsamp(fileDir+'/%s'%(base))
            PPG.append(sample[0][:,1])

    PPG = np.asarray(PPG)
    return PPG

## Segment Data


```slidingWindow()``` returns a generator that iterates through the input sequence.

In [0]:
def slidingWindow(sequence,winSize=2048,step=256):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    numOfChunks = ((len(sequence)-winSize)//step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        yield sequence[i:i+winSize]

### PPG

Here the PPG signal is segmented using the ```slidingWindow``` function

The returned signal **p** is ready to be plotted

In [0]:
def segment_PPG(PPG):
  p = []

  for i in range(len(PPG)):
    ppg = slidingWindow(PPG[i], winSize = 2048, step = 256)
    for j in ppg:
      p.append(j)
    
  p = np.asarray(p)
  return p

##Plot and Save PPG

In [0]:
def plot_and_save(exercise, p, ds_factor):
  pixel_size = 386
  my_dpi = 96
  fs = 256.0

  for i in range(len(p)):
    fig = plt.figure( figsize = (pixel_size/my_dpi, pixel_size/my_dpi), dpi=my_dpi )
    ax = fig.add_subplot(111)
    ax.plot(p[i, ::ds_factor])
    # Bound the plot to prevent normalising of signals
    ax.scatter(0, -30, c='w') # min value of all signals
    ax.scatter(0, 2820, c='w') # max value of all signals

    # Remove all extraneous graph elements
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)

    # Save the figure and close
    plt.savefig('./Data/PPG_'+str(int(fs//ds_factor))+'Hz/'+
                str(exercise)+'/'+str(i)+'.jpg', bbox_inches = 'tight', pad_inches = 0)
    plt.close()


## Main - function calls

The functions above are called to segment, plot and save your PPG data into folders ready to be used in Transfer Learning for HAR

In [0]:
# Exercises in dataset
exercise = ['High', 'Low', 'Run', 'Walk']
# Original sampling frequency
fs = 256.0
# Downsampling Factor
# 256Hz 30Hz 15Hz 12Hz 11Hz 10Hz 9Hz 8Hz 5Hz 1Hz
dwns_factor = [fs//256.0, fs//30.0, fs//15.0, fs//12.0, fs/11.0, fs//10.0, fs//9.0, fs//8.0, fs//5.0, fs//1.0]
# File Directory for data
fileDir='./Data/wrist'


for exer in exercise:
    # Load Data - only need to load once per exercise
    PPG = load_data(fileDir, exer)
    # Downsample and plot
    for d in dwns_factor:
        d = int(d)
        # Segment data
        ppg = segment_PPG(PPG)
        # Plot and save - ready for Transfer Learning
        plot_and_save(exer, ppg, d)

./Data/wrist/s2_high_resistance_bike
./Data/wrist/s1_high_resistance_bike
./Data/wrist/s3_high_resistance_bike
./Data/wrist/s1_low_resistance_bike
./Data/wrist/s3_low_resistance_bike
./Data/wrist/s2_low_resistance_bike
./Data/wrist/s5_low_resistance_bike
./Data/wrist/s6_low_resistance_bike
./Data/wrist/s3_run
./Data/wrist/s4_run
./Data/wrist/s5_run
./Data/wrist/s6_run
./Data/wrist/s8_run
./Data/wrist/s1_walk
./Data/wrist/s2_walk
./Data/wrist/s3_walk
./Data/wrist/s6_walk
./Data/wrist/s9_walk
./Data/wrist/s8_walk
